### Chapter 44. 생존분석(Survival analysis)

#### **개요**

**생존자료(survival data)**  : 관심종료점(endpoint, 주로 사망)에 도달할 때까지 걸린 시간과 관련된 자료

- 생존자료의 특성

   1) 주된 관심사항은 환자가 종료점에 도달했는지 여부가 아닌 종료점에 도달할 때까지 걸린 시간임(=length of time)  
    > 예) 간경변증으로 입원한 환자가 생존한 기간
   2) 자료가 중도절단(censored) 될 수 있음.


- 환자들이 종료점에 도달했는지 여부를 `로지스틱 회귀`분석으로 평가하거나, 집단간 평균시간을 비교하기 위해 `독립t검정`을 사용하는 것은 **자료의 일부가 중도절단되었다는 정보를 사용하지 않기 때문에 잘못된 분석임.**
- 따라서 이 문제를 다루기 위해 생존 분석이라는 일련의 통계적인 방법들이 개발됨.

**생존자료 예시**  
생존자료, 즉 개인의 생존시간을 조사한 자료는 사건-시간자료(event time data)라고도 함.  
사건-시간자료에 관해 엑셀파일로 정리하면 아래와 같은 형태임.  
6명에 대한 자료로, 기준시점인 수술일, 관심종료점인 사망일이 YYYY-MM-DD형태로 입력됨.  
![1](1.png)  
생존분석을 위해서는 위의 원자료를 아래와 같은 형태로 재구축해야 함.  
원자료와 차이점을 보면  
    1) `FU_days`를 추가하여 수술일부터 마지막 방문 혹은 사망일까지의 시간(일)을 계산함.  
    2) `Event` 1=관심사건(질병X)로 인한 사망, 0=중도절단 으로 D_reason과 V_reason을 정리함.  
    3) `Event_c`을 만들어 질병X로 인한 사망위험을 분석할 때 질병Y로 인한 사망은 중도 절단이 아닌 X의 경쟁사건(competeting event)으로 간주하면서 분석함.   
![2](2.png) 

----

#### **중도절단자료(cesored data)**

- 생존시간(survival time) : 시작점에 해당하는 기준시점부터 환자가 관심종료점(사망 혹은 재발 등)에 도달한 시점 사이의 시간
- 연구기간 도중 **실제 종료점에 도달한 시점에 대해서는 알 수 없고**, 종료점과 무관한 시점만 알 수 있는 경우가 있음. → **오른쪽이 중도절단된 자료(right-censored data)**
- 이 환자들에 대해서 최소한 이들이 마지막으로 추적된 시점까지는 종료점에 도달하지 않았다는 것을 알고 있기 때문에 종료점이 발생한다고 하더라도 종료점 발생 시점은 적어도 마지막 관찰시점 이후일 것이라는 정보를 사용해야 함.  
> **예**)  항암제 신약 임상시험을 진행하는 경우, `관심종료점`이 `암이 진행(progression)되는 시점`이라고 할 때  
  환자가 **(1) 임상시험 도중 관해가 되거나, (2) 관해가 되고 있는 상태에서 연구가 종료되거나, (3) 암이 진행되기 전에 연구 참여를 철회하거나, (4) 추적기간이 종료되기 전에 다른 원인으로 사망할 경우** 관심종료점을 
  관찰할 수 없음.  
      > `관해`:  암 치료에서 증상이 완화되거나 사라진 상태를 말함. 완치 대신 임상에서 해당 용어를 사용하는 이유는 완치는 재발 가능성이 더 이상 없다는 것을 의미하는데 이를 의학적으로 증명하기 어렵기 때문임.

- 연구 시작시점 이전 상황에 대한 추적이 불가능한 생존자료는 **왼쪽이 중도절단된 자료(left-censored data)** 라고 함.  
> **예**)
  추적조사 시점이 HIV에 감염된 것으로 확인된 시점이라고 한다면, 환자들이 실제 HIV에 감염된 때는 해당 시점 이전일 것임. 

- 참고로 양쪽 모두 중도절단된 자료는 **구간 중도절단 자료(interval censored data)** 라고 함.

----

#### **생존자료의 표현**

**1. 생존자료를 표현하기 위해 각 환자별 수평선을 그림.**  

<img src="6.jpg" alt="6" width="800"/>

-  6명 환자들의 생존시간에 대한 원자료를 실제 관찰시간을 x축으로 사용해 표현한 것임.
- `선`은 `생존시간`으로 왼쪽으로 시작해서 오른쪽으로 그림.  
- `흰 원`은 `사건 발생`을, `삼각형`은 연구 탈락이나 연구참여 동의철회 등으로 인한 `추적 실패`, `검은 점`은 `연구종료`를 의미함.  


<img src="3.jpg" alt="3" width="800"/> 

- 원자료를 생존기간에 대해 재구축한 자료를 생존시간(year)을 x축으로 사용해 표현  
- `흰 원`은 `관심사건`, `검은 점`은 `중도절단`으로 표현함.

- 자료를 요약한 그림이 아니기 때문에 **환자들의 전반적인 생존상황에 관한 윤곽을 파악하는데에 한계가 있음.**

**2. 카플란 마이어(Kaplan-Meier) 방법 생존곡선(survival curve)**  

카플란 마이어(Kaplan-Meier) 방법  
- 생존함수(t시점 이상 생존할 확률) $\displaystyle S(t) = P(T \ge t)=1-$$\int\limits_0^t$$f(x)dx$ 를 추정하는 방법 
- 누적한계추정법(product-limit estimation method)라고도 함.  

1) 발생시간을 크기 순으로 정렬 ($t_0$: 기저시점, 주로 0, $t_{k+1}=\infty$)   
2) 각 구간 $[t_i, t_{i+1}), i=0,\ldots,k$별로 다음 인원수를 구함
    -  $n_i$ = 아직 event가 발생하지 않은 것이 확인된 인원 수, 
    -  $d_i$=해당 구간 내에서 사건이 발생한 인원수 


3) $\displaystyle \frac{d_i}{n_i}$는 각 구간내 사건발생율, $\displaystyle 1-\frac{d_i}{n_i}$는 각 구간내 생존비율이 됨.

4) 시점 t에서 누적생존확률은 t시점 이전의 각 구간별 생존비율을 누적해서 곱한 값, 
$$
 \hat{S}(t) = \prod_{t_i<t}\bigg(1-\frac{d_i}{n_i}\bigg)
$$

5) 시간 t에 따른 $\hat{S}(t)$값의 그림을 누적생존곡선(cumulative survival curve)이라고 함.

- 생존곡선은 기준 시점 이후의 모든 시점에서 개인이 종료점 사건을 경험하지 않을 누적확률이며 이를 누적생존확률이라고 함.  

<img src="5.jpg" alt="5" width="500"/>  

- 생존곡선은 x축(시간)의 값이 0(기저시점)일 때의 y축(누적생존확률)의 값 1.0(100%)부터 시작해 시간이 지남에 따라 누적생존확률 0을 향해 연속적으로 감소하는 곡선의 형태가 됨.
- 실제로 관찰된 자료는 종료점 사건이 발생하는 시점에서만 누적생존확률이 변하기 때문에 계단식으로 내려감.

- 종료점에 관한 누적 발생률로 표현할 수도 있음.  
- 발생률은 `1-(누적생존확률)`로 계산하며, 따라서 누적발생률 곡선(cumulative incidence curve)는 생존곡선의 역형태가 됨.  

<img src="7.png" alt="7" width="300"/><img src="8.png" alt="8" width="300"/>  

ref : https://blog.zarathu.com/posts/2020-10-29-survivalpractice/  

- 이 둘 그림에 포함된 정보는 동일하지만, 종료점이 `희귀사건`인 경우에는 누적생존확률이 연구 전반에 걸쳐 높게 유지되기 때문에 곡선의 형태를 최대한 자세히 묘사할 수 있는 `누적발생률 곡선을 사용하는 것이 효과적`임.

**3. 생명표(life table) 이용하기**  
- 누적생존확률을 계산하는 방법에는 생명표를 이용할 수 있음.
- 각 개인이 종료점에 도달한 정확한 시간은 알 수 없지만, 일정 구간 내(예. 연도별 등) 종료점이 발생한 사람들의 빈도는 알 수 있는 경우에 사용할 수 있음.
- 보험수리적 분석을 하는 보험계리학에서 많이 사용함.

----

#### **분석결과의 요약 및 제시**

- 생존자료를 요약하는 방법들

  1) 생존곡선 상의 특정 시점에 대한 누적생존확률 및 이에 관한 신뢰구간을 제시함.
  
  2) 종료점에 도달하는 중앙 생존시간(median survival time, 환자들 중 50%가 질병에 걸리게 되는 시간) 및 해당 신뢰구간을 제시함.

- 두 집단 간 생존곡선 비교결과를 제시하는 방법 (`p274`)  
  1) 집단 간에 생존곡선이 일관되게 벌어져 있으면(교차하지 않으면) 전반적인 생존분석 결과를 제시함.
  > **예**) `A 집단`의 생존곡선이`B 집단`에 비해 일관성있게 낮으면, `B 집단`에 비해 `A 집단`에서 event들이 더 일찍 발생했고, 이로 인해 모든 추적조사 시점에서 `A 집단`의 누적생존확률이 `B 집단`보다 더 작다는 것을 의미함.
 
 2) 특정 누적생존확률 값을 선택해서 이에 관한 생존시간을 각 집단별로 제시함.
     - 누적생존확률 값은 주로 50%를 사용하며, 이에 해당하는 생존시간이 중앙 생존시간임.
     - 평균생존시간(mean survival time)은 개념적으로 생존곡선 아래 부분의 면적에 해당되는 값임. 
     - 자료 내 가장 긴 추적기간에 해당하는 사람의 자료가 중도절단 자료 이면, 평균 생존시간이 정의되지 않음.
     - 평균 생존시간은 중도절단이 포함된 자료에 대한 생존분석에서는 거의 사용되지 않음.


 3) 특정 시점을 선택해서 해당 시점의 누적생존확률을 각 집단별로 제시함.  
    - 주로 각 집단의 누적생존확률 추정값들이 너무 커서 중앙생존시간이 계산되지 않는 경우 사용  
      
참고) 각 시점별 누적생존확률의 95% 신뢰구간
$$\hat{S}(t) \pm 1.96\times SE[\hat{S}(t)]$$

----

#### **생존곡선의 비교**

집단 간 누적생존확률을 비교하고자 하는 경우, 
- 먼저 집단별 생존곡선을 그려서 종료점에 도달하는 율이 집단들 간에 서로 다른지 여부를 눈으로 확인
- 검정방법으로 `로그-순위 검정`과 `회귀모형`을 이용하는 방법 등이 있음.

**로그-순위 검정(log-rank test)**
- 집단들 간에 생존시간은 유의한 차이가 없다는 귀무가설을 검정하기 위한 비모수적 검정방법
- 종료점이 발생한 모든 시점별로 `기대 사건수`와 `관찰 사건수 간 누적차이`를 검정통계량으로 사용.
- 누적생존확률에 영향을 미치는 하나의 요인(집단을 나타내는 요인)외 다른 추가적인 요인들의 독립적인 효과를 함께 평가하는데 사용할 수 없음.

**로그-순위 검정 단계별로 알아보기**

귀무가설 : $H_0 : S_A(t) = S_B(t)$

1) 두 집단의 자료를 합함.  
2) $r$개 발생시간을 크기 순으로 정렬한 뒤 각 구간 $[t_i, t_{i+1})$내에서 공통사건발생률(common event rate)$\displaystyle \frac{d_i}{n_i}$를 계산  
\begin{align*}
   n_i &= \mbox{집단 구분 없이)해당 구간 시작시점에서의 위험대상 인원 수, }\\
   d_i &= \mbox{해당 구간 내 관찰 사건 수  }\\
   n_{Ai}&+n_{Bi}=n_i,\quad d_{Ai}+d_{Bi}=d_i
\end{align*}
   
3) 귀무가설 하에 집단별로, 각 구간별 기대되는 기대사건수(expected number of events) 
$$
e_{Ai} = n_{Ai} \times \frac{d_i}{n_i},\quad    e_{Bi} = n_{Bi} \times \frac{d_i}{n_i}
$$  
카이제곱 검정의 기대빈도를 계산하는 방법과 동일한 개념임.

4) $U_L = $$\sum_{i=1}^{r} (d_{Ai} - e_{ai})$은 근사적으로 평균이 0이고 분산이 $\displaystyle V_L = \sum_{i=1}^{r} \frac{n_{Ai}n_{Bi}}{n_i^2(n_i-1)}\times d_i(n_i-d_i)$인 정규분포에 따름.

5) 검정통계량 $\displaystyle L=\frac{U_L^2}{V_L}\stackrel{\cdot}{\sim}\chi^2_1$

**Wilcoxon 검정(Breslow 검정)**

-  $\displaystyle U_W = \sum_{i=1}^{r} n_i(d_{Ai}-e_{Ai})$와 $\displaystyle V_W=\sum_{i=1}^{r} \frac{n_{Ai}n_{Bi}}{(n_i-1)}d_i(n_i-d_i)$
- 검정통계량 $W= \frac{U_w^2}{V_w}\stackrel{\cdot}{\sim}\chi^2_1$

**로그-순위검정와 Wilcoxon 검정(Breslow 검정) 차이점**

- 로그-순위검정은 모든 시점에 걸쳐 $(d_{Ai}-e_{Ai})$에 동일한 크기의 가중값$(\displaystyle\frac{1}{\sqrt{V_L}})$을 부여하는 검정통계량을 사용
- Wilcoxon검정은 표본수가 많은 초기에 로그-순위 검정보다 더 큰 가중값$(\displaystyle\frac{n_i}{\sqrt{V_L}})$을 부여하는 검정통계량을 사용
- Wilcoxon검정은 집단 간 생존곡선이 연구 후반보다 연구 초반에 상대적으로 더 많은 차이가 나는 상황에 민감한 방법(통계적 유의성을 더 쉽게 보여주는 방법)임.  
  - 두 집단의 생존곡선이 연구 후반부에서만 일부 겹칠 때 효과적으로 사용될 수 있음.  


- 로그-순위 검정은 일반적으로 집단 간 누적생존확률의 차이를 좀 더 잘 평가해주는 방법 
  - 두 집단의 생존곡선이 전체적으로 겹치지 않을 때 사용하는 것이 좋음.